In [1]:
import pandas as pd

In [2]:
# df_sg_explode = pd.read_parquet('./data/clean/sg_explode.parquet.gzip')
df_sg_explode = pd.read_parquet('https://github.com/xaviac/storage__PI_MLOp/raw/main/data/clean/sg_explode.parquet.gz')


# df_ur = pd.read_parquet('./data/clean/user_reviews.parquet.gzip')
df_ur = pd.read_parquet('https://github.com/xaviac/storage__PI_MLOp/raw/main/data/clean/user_reviews.parquet.gz')

# df_ui = pd.read_parquet('./data/clean/users_items.parquet.gzip')
df_ui = pd.read_parquet('https://github.com/xaviac/storage__PI_MLOp/raw/main/data/clean/users_items.parquet.gz')

In [3]:
union_ui_sg = pd.merge(df_ui, df_sg_explode, left_on='item_name', right_on='title', how='inner')

In [4]:
union_ui_sg

,user_id,item_id,item_name,playtime_forever_hours,genres,title,price,id,developer,release_year
0,76561197970982479,10,Counter-Strike,0.10,Action,Counter-Strike,9.99,10,Valve,2000
1,doctr,10,Counter-Strike,1.55,Action,Counter-Strike,9.99,10,Valve,2000
2,corrupted_soul,10,Counter-Strike,1.80,Action,Counter-Strike,9.99,10,Valve,2000
3,WeiEDKrSat,10,Counter-Strike,5.47,Action,Counter-Strike,9.99,10,Valve,2000
4,death-hunter,10,Counter-Strike,104.58,Action,Counter-Strike,9.99,10,Valve,2000
...,...,...,...,...,...,...,...,...,...,...
6730513,danebuchanan,485270,Arachnophobia,0.12,Casual,Arachnophobia,4.99,485270,Unknown,2015
6730514,danebuchanan,485270,Arachnophobia,0.12,Simulation,Arachnophobia,4.99,485270,Unknown,2015
6730515,danebuchanan,485270,Arachnophobia,0.12,Psychological Horror,Arachnophobia,4.99,485270,Unknown,2015
6730516,danebuchanan,485270,Arachnophobia,0.12,Horror,Arachnophobia,4.99,485270,Unknown,2015


In [5]:
group_by_year_genres = pd.DataFrame(union_ui_sg.groupby(['release_year', 'genres'])['playtime_forever_hours'].sum().reset_index())

In [6]:
group_by_year_genres

,release_year,genres,playtime_forever_hours
0,1983,Action,59.74
1,1983,Adventure,59.74
2,1983,Casual,59.74
3,1984,Action,6.40
4,1984,Adventure,6.40
...,...,...,...
654,2018,Adventure,0.35
655,2018,Early Access,102.14
656,2018,Indie,38.80
657,2018,RPG,63.69


In [7]:
# Guardar el dataframe en un archivo csv en la carpeta functions
group_by_year_genres.to_csv('./data/functions/group_by_year_genres.csv.gz', compression='gzip', index=False, encoding='utf-8')

In [8]:
# Guardar el dataframe en un archivo parquet en la carpeta functions
group_by_year_genres.to_parquet('./data/functions/group_by_year_genres.parquet.gz', compression='gzip', index=False)

In [9]:
def PlayTimeGenre(genero: str):
     # Filtramos el dataframe para el generes
    df_genres = group_by_year_genres[group_by_year_genres['genres'].str.contains(genero, case=False, na=False)]

    # Agrupamos release_year y sumamos playtime_forever_hours
    total_by_year = df_genres.groupby('release_year')['playtime_forever_hours'].sum()
    # Se encuentra el año con más horas jugadas
    top_year = total_by_year.idxmax()

    return {f'Año de lanzamiento con más horas jugadas para {genero}: {top_year}'}


In [10]:
PlayTimeGenre('Strategy')


{'Año de lanzamiento con más horas jugadas para Strategy: 2015'}

In [11]:
group_by_user_genres_year = pd.DataFrame(union_ui_sg.groupby(['user_id', 'genres', 'release_year'])['playtime_forever_hours'].sum().reset_index())

In [12]:
group_by_user_genres_year

,user_id,genres,release_year,playtime_forever_hours
0,--000--,Action,2009,88.81
1,--000--,Action,2010,0.37
2,--000--,Action,2011,97.61
3,--000--,Action,2012,1788.14
4,--000--,Action,2013,6.05
...,...,...,...,...
3317493,zzzmidmiss,Sports,2010,3.23
3317494,zzzmidmiss,Strategy,2010,0.17
3317495,zzzmidmiss,Strategy,2011,14.22
3317496,zzzmidmiss,Strategy,2012,0.75


In [13]:
# Eliminar valores < 0 de playtime_forever_hours
group_by_user_genres_year = group_by_user_genres_year[group_by_user_genres_year['playtime_forever_hours'] > 1]


In [14]:
# Guardar el dataframe en un archivo csv en la carpeta functions
group_by_user_genres_year.to_csv('./data/functions/group_by_user_genres_year.csv.gz', compression='gzip', index=False, encoding='utf-8')

In [15]:
# Guardar el dataframe en un archivo parquet en la carpeta functions
group_by_user_genres_year.to_parquet('./data/functions/group_by_user_genres_year.parquet.gz', compression='gzip', index=False)

In [16]:
def UserForGenre(genero: str):
    # Filtrar el DataFrame por el género dado

    generes = group_by_user_genres_year[group_by_user_genres_year['genres'].str.contains(genero)] 
    #Agrupo por usuario y sumo cantidad de horas jugadas
    hours = generes.groupby('user_id')['playtime_forever_hours'].sum().reset_index()


    # Encontrar el usuario con más horas jugadas para ese género
    playtime_by_user = hours.loc[hours['playtime_forever_hours'].idxmax()]['user_id']


    # Filtrar el DataFrame por el usuario con más horas jugadas en ese género
    top_user = generes[generes['user_id'] == playtime_by_user]
    total_hours_year = top_user.groupby('release_year')['playtime_forever_hours'].sum().reset_index()

    user_hours_years = [{'Año': int(row['release_year']), 'Horas': int(row['playtime_forever_hours'])} for _, row in total_hours_year.iterrows()]
    
    return {f'Usuario con más horas jugadas para {genero}': playtime_by_user, 'Horas jugadas': user_hours_years}

In [17]:
UserForGenre('Action')

{'Usuario con más horas jugadas para Action': 'shinomegami',
 'Horas jugadas': [{'Año': 1988, 'Horas': 2},
  {'Año': 1996, 'Horas': 19},
  {'Año': 1997, 'Horas': 1},
  {'Año': 1998, 'Horas': 61},
  {'Año': 1999, 'Horas': 5},
  {'Año': 2000, 'Horas': 179},
  {'Año': 2001, 'Horas': 40},
  {'Año': 2003, 'Horas': 8879},
  {'Año': 2004, 'Horas': 2},
  {'Año': 2005, 'Horas': 5},
  {'Año': 2006, 'Horas': 34},
  {'Año': 2007, 'Horas': 1139},
  {'Año': 2008, 'Horas': 3},
  {'Año': 2009, 'Horas': 3496},
  {'Año': 2010, 'Horas': 156},
  {'Año': 2011, 'Horas': 1489},
  {'Año': 2012, 'Horas': 308},
  {'Año': 2013, 'Horas': 2469},
  {'Año': 2014, 'Horas': 1534},
  {'Año': 2015, 'Horas': 3033},
  {'Año': 2016, 'Horas': 709},
  {'Año': 2017, 'Horas': 2}]}

In [18]:
df_sg = pd.read_parquet('https://github.com/xaviac/storage__PI_MLOp/raw/main/data/clean/steam_games.parquet.gz')

In [19]:
df_sg

,genres,title,price,id,developer,release_year
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,0.00,643980,Secret Level SRL,2018
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,0.00,670290,Poolians.com,2017
3,"[Action, Adventure, Casual]",弹炸人2222,0.99,767400,彼岸领域,2017
4,"[Action, Indie, Casual, Sports]",Log Challenge,2.99,773570,Unknown,2015
...,...,...,...,...,...,...
32128,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,1.99,773640,"Nikita ""Ghost_RUS""",2018
32129,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,4.99,733530,Sacada,2018
32130,"[Indie, Racing, Simulation]",Russian Roads,1.99,610660,Laush Dmitriy Sergeevich,2018
32131,"[Casual, Indie]",EXIT 2 - Directions,4.99,658870,"xropi,stev3ns",2017


In [20]:
union_ur_sg = pd.merge(df_ur, df_sg, left_on='item_id', right_on='id', how='inner')

In [21]:
union_ur_sg

,user_id,item_id,recommend,sentiment_analysis,genres,title,price,id,developer,release_year
0,76561197970982479,1250,True,2,[Action],Killing Floor,19.99,1250,Tripwire Interactive,2009
1,death-hunter,1250,True,2,[Action],Killing Floor,19.99,1250,Tripwire Interactive,2009
2,DJKamBer,1250,True,0,[Action],Killing Floor,19.99,1250,Tripwire Interactive,2009
3,diego9031,1250,True,1,[Action],Killing Floor,19.99,1250,Tripwire Interactive,2009
4,76561198081962345,1250,True,1,[Action],Killing Floor,19.99,1250,Tripwire Interactive,2009
...,...,...,...,...,...,...,...,...,...,...
53983,llDracuwulf,307130,True,1,"[Action, Adventure, Indie]",Asteria,4.99,307130,Legend Studio,2014
53984,ChrisCoroner,209120,True,2,[Action],Street Fighter X Tekken,29.99,209120,"Capcom U.S.A., Inc.",2012
53985,MeloncraftLP,220090,True,1,"[Adventure, Indie]",The Journey Down: Chapter One,8.99,220090,SkyGoblin,2013
53986,MeloncraftLP,262850,True,1,"[Adventure, Indie]",The Journey Down: Chapter Two,19.99,262850,SkyGoblin,2014


In [22]:
union_ur_sg = union_ur_sg.drop(columns=['item_id', 'id','genres'], axis=1)

In [23]:
union_ur_sg

,user_id,recommend,sentiment_analysis,title,price,developer,release_year
0,76561197970982479,True,2,Killing Floor,19.99,Tripwire Interactive,2009
1,death-hunter,True,2,Killing Floor,19.99,Tripwire Interactive,2009
2,DJKamBer,True,0,Killing Floor,19.99,Tripwire Interactive,2009
3,diego9031,True,1,Killing Floor,19.99,Tripwire Interactive,2009
4,76561198081962345,True,1,Killing Floor,19.99,Tripwire Interactive,2009
...,...,...,...,...,...,...,...
53983,llDracuwulf,True,1,Asteria,4.99,Legend Studio,2014
53984,ChrisCoroner,True,2,Street Fighter X Tekken,29.99,"Capcom U.S.A., Inc.",2012
53985,MeloncraftLP,True,1,The Journey Down: Chapter One,8.99,SkyGoblin,2013
53986,MeloncraftLP,True,1,The Journey Down: Chapter Two,19.99,SkyGoblin,2014


In [24]:
# Recorrer 'title' buscando '&amp;' verificando que fue corregido
union_ur_sg['title'][union_ur_sg['title'].apply(lambda x: '&amp;' in x)]

Series([], Name: title, dtype: object)

In [25]:
# Recorrer con un for y sustituir dentro de la columna title '&amp;' por '&'
for title in union_ur_sg['title'][union_ur_sg['title'].apply(lambda x: '&amp;' in x)]:
    if isinstance(title, str):
        union_ur_sg['title'].replace(title, title.replace('&amp;', '&'), inplace=True)

In [26]:
# Recorrer 'developer' buscando '&amp;' verificando que fue corregido
union_ur_sg['developer'][union_ur_sg['developer'].apply(lambda x: '&amp;' in x)]

Series([], Name: developer, dtype: object)

In [27]:
# Recorrer con un for y sustituir dentro de la columna developer '&amp;' por '&'
for developer in union_ur_sg['developer'][union_ur_sg['developer'].apply(lambda x: '&amp;' in x)]:
    if isinstance(developer, str):
        union_ur_sg['developer'].replace(developer, developer.replace('&amp;', '&'), inplace=True)

In [28]:
union_ur_sg.to_csv('./data/functions/union_ur_sg.csv.gz', compression='gzip', index=False, encoding='utf-8')

In [29]:
union_ur_sg.to_parquet('./data/functions/union_ur_sg.parquet.gz', compression='gzip', index=False)

In [30]:
def UsersRecommend(year: int):
    # Filtramos el DataFrame donde la columna 'release_year' es igual a year, la columna 'recommend' es True y la columna 'sentiment_analysis' tiene valores 1 o 2. 
    filter = union_ur_sg['title'][(union_ur_sg['release_year'] == year) & (union_ur_sg['recommend'] == True) & (union_ur_sg['sentiment_analysis'].isin([1, 2]))].value_counts().reset_index().head(3)
    
    return [{f'Puesto {i + 1}: {row['title']}'} for i, row in filter.iterrows()]

In [31]:
UsersRecommend(2000)

[{'Puesto 1: Counter-Strike'},
 {'Puesto 2: Deus Ex: Game of the Year Edition'},
 {'Puesto 3: Ricochet'}]

In [32]:
def UsersNotRecommend(year: int):
    # Se filtra las filas del DataFrame donde la columna 'release_year' es igual a year, la columna 'recommend' es False y la columna 'sentiment_analysis' con valor 0
    filter = union_ur_sg['title'][(union_ur_sg['release_year'] == year) & (union_ur_sg['recommend'] == False) & (union_ur_sg['sentiment_analysis']==0)].value_counts().reset_index().head(3)
    
    return [{f'Puesto {i + 1}: {row['title']}'} for i, row in filter.iterrows()]

In [33]:
UsersNotRecommend(2011)

[{'Puesto 1: APB Reloaded'},
 {'Puesto 2: No More Room in Hell'},
 {'Puesto 3: Terraria'}]

In [34]:
def sentiment_analysis(year: int):
    # Filtrar el DataFrame para el año
    filter = union_ur_sg[union_ur_sg['release_year'] == year]
    #Cuenta los comentarios positivos
    positives = filter[filter['sentiment_analysis']==2]['sentiment_analysis'].count()
    # Cuenta los comentarios negativos
    negatives = filter[filter['sentiment_analysis']==0]['sentiment_analysis'].count()
    # Cuenta los comentarios neutrales
    neutrals = filter[filter['sentiment_analysis']==1]['sentiment_analysis'].count()
    # Devolver conteos en un diccionario

    return {'Negative': int(negatives), 'Positive': int(positives), 'Neutral': int(neutrals)}

In [35]:
sentiment_analysis(2017)

{'Negative': '177', 'Positive': '986', 'Neutral': '941'}